<a href="https://colab.research.google.com/github/parasgulati8/Web-Scraper/blob/master/scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import glob
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup as bs
# import urllib3
import requests
# import json
# from urllib.request import urlopen, urlretrieve, quote
# from openpyxl import Workbook
import re

In [0]:
page = requests.get('http://www.padctn.org/prc/property/1/card/1/historical')

In [0]:
soup = bs(page.text, 'html.parser')
# u = urlopen(site)

In [7]:
soup.h2.text

'Historical Details - Map & Parcel 001 00 0 001.00'

In [18]:
' '.join(parcel.split()[6:])

'001 00 0 001.00'

In [19]:
url = 'http://www.padctn.org/prc/property/1/card/1/historical'
page = requests.get(url)
soup = bs(page.text, 'html.parser')
parcel = soup.h2.text
parcel = ' '.join(parcel.split()[6:])
df = pd.read_html(requests.get(url,  headers={'User-agent': 'Mozilla/5.0'}).text,
                      attrs={"class":"table table-condensed table-bordered"})[0]

df['Parcel'] = [parcel]*len(df)
df

,Sale Date,Sale Price,Deed Type,Deed Book & Page,Parcel
0,12/05/2006,"$7,500",DEED OF CORRECTION,20110805-0060547,001 00 0 001.00
1,12/05/2006,"$7,500",WARRANTY DEED,20070109-0003943,001 00 0 001.00
2,09/15/1965,$0,WARRANTY DEED,00000137-0000216,001 00 0 001.00


In [0]:
table = soup.table

In [0]:
table

<table class="table table-condensed table-bordered">
<tr><th>Sale Date</th><th>Sale Price</th><th>Deed Type</th><th>Deed Book &amp; Page</th></tr>
<tr><td>12/05/2006</td><td>$7,500</td><td>DEED OF CORRECTION</td><td>20110805-0060547</td></tr>
<tr><td>12/05/2006</td><td>$7,500</td><td>WARRANTY DEED</td><td>20070109-0003943</td></tr>
<tr><td>09/15/1965</td><td>$0</td><td>WARRANTY DEED</td><td>00000137-0000216</td></tr>
</table>

In [0]:
rows = table.find_all('tr')

In [0]:
rows

[<tr><th>Sale Date</th><th>Sale Price</th><th>Deed Type</th><th>Deed Book &amp; Page</th></tr>,
 <tr><td>12/05/2006</td><td>$7,500</td><td>DEED OF CORRECTION</td><td>20110805-0060547</td></tr>,
 <tr><td>12/05/2006</td><td>$7,500</td><td>WARRANTY DEED</td><td>20070109-0003943</td></tr>,
 <tr><td>09/15/1965</td><td>$0</td><td>WARRANTY DEED</td><td>00000137-0000216</td></tr>]

In [0]:
tuples = []
for tr in rows:
  td = tr.find_all('td')
  th = tr.find_all('th')
  print(th)
  print([i.text for i in th])
  # row = [i.text for i in th]
  # tuples.append([i.text for i in th])
  # if row:
    # print(row)
    # tuples.append(row)

[<th>Sale Date</th>, <th>Sale Price</th>, <th>Deed Type</th>, <th>Deed Book &amp; Page</th>]
['Sale Date', 'Sale Price', 'Deed Type', 'Deed Book & Page']
[]
[]
[]
[]
[]
[]


In [0]:
tuples

[['12/05/2006', '$7,500', 'DEED OF CORRECTION', '20110805-0060547'],
 ['12/05/2006', '$7,500', 'WARRANTY DEED', '20070109-0003943'],
 ['09/15/1965', '$0', 'WARRANTY DEED', '00000137-0000216']]

In [0]:
import pandas as pd

In [0]:
url = 'http://www.padctn.org/prc/property/1/card/1/historical'
dfs = pd.read_html(requests.get(url,  headers={'User-agent': 'Mozilla/5.0'}).text,
                      attrs={"class":"table table-condensed table-bordered"})

In [0]:
dfs[1]

,Year,Land Use Code,Building,Yard Items,Land Value,Category,Total
0,2017,0AR - AG/RES,$0,$0,"$10,621",INF,"$10,621"
1,2017,0AR - AG/RES,$0,$0,"$10,621",ROLL,"$10,621"
2,2013,0RA - RES/AGR,$0,$0,"$8,565",INF,"$8,565"
3,2013,0RA - RES/AGR,$0,$0,"$8,565",ROLL,"$8,565"
4,2009,0RA - RES/AGR,$0,$0,"$6,909",INF,"$6,909"
5,2009,0RA - RES/AGR,$0,$0,"$6,909",ROLL,"$6,909"
6,2008,0RA - RES/AGR,$0,$0,"$5,572",JUNE,"$5,572"
7,2008,0RA - RES/AGR,$0,$0,"$5,572",ROLL,"$5,572"
8,2005,E80 - EXE,$0,$0,"$25,800",ROLL,"$25,800"
9,2001,E80 - EXE,$0,$0,"$22,800",ROLL,"$22,800"


In [23]:
url = 'http://www.padctn.org/prc/property/1/card/1/historical'
page = requests.get(url)
soup = bs(page.text, 'html.parser')
parcel = soup.h2.text
parcel = ' '.join(parcel.split()[6:])
df = pd.read_html(requests.get(url,  headers={'User-agent': 'Mozilla/5.0'}).text,
                      attrs={"class":"table table-condensed table-bordered"})[0]

df['Parcel'] = [parcel]*len(df)
i = 2
while True:
  url = 'http://www.padctn.org/prc/property/' + str(i) + '/card/' + str(i) + '/historical'
  page = requests.get(url)
  soup = bs(page.text, 'html.parser')
  parcel = soup.h2.text
  parcel = ' '.join(parcel.split()[6:])
  
  dfs = pd.read_html(requests.get(url,  headers={'User-agent': 'Mozilla/5.0'}).text,
                        attrs={"class":"table table-condensed table-bordered"})
  dfs[0]['Parcel'] = [parcel]*len(dfs[0])
  print(i, end=' ')
  if np.any(dfs[0]) or np.any(dfs[1]):
    df = pd.concat([df, dfs[0]] )
    i = i+1
  else:
    print ('Table Empty, Exiting...')
    print (table)
    break
  if i % 10 == 1:
    print()
  # time.sleep(3)

2 3 4 5 6 7 8 9 10 
11 12 13 14 15 16 17 18 19 20 
21 22 23 24 25 26 27 28 29 30 
31 32 33 34 35 36 37 38 39 40 


KeyboardInterrupt: ignored

In [24]:
df

,Sale Date,Sale Price,Deed Type,Deed Book & Page,Parcel
0,12/05/2006,"$7,500",DEED OF CORRECTION,20110805-0060547,001 00 0 001.00
1,12/05/2006,"$7,500",WARRANTY DEED,20070109-0003943,001 00 0 001.00
2,09/15/1965,$0,WARRANTY DEED,00000137-0000216,001 00 0 001.00
0,05/17/1983,"$12,500",WARRANTY DEED,00006058-0000985,001 00 0 002.00
1,01/06/1972,$0,WARRANTY DEED,00005797-0000390,001 00 0 002.00
...,...,...,...,...,...
2,10/10/1974,"$41,300",WARRANTY DEED,00006409-0000762,002 00 0 023.00
3,01/22/1953,$0,WARRANTY DEED,00002539-0000482,002 00 0 023.00
0,08/30/1982,"$21,682",WARRANTY DEED,00005931-0000475,002 00 0 024.00
1,10/14/1981,"$40,000",WARRANTY DEED,00005816-0000873,002 00 0 024.00


In [0]:
import numpy as np


In [0]:
url = 'http://www.padctn.org/prc/property/1000000/card/1000000/historical'
t = pd.read_html(requests.get(url,  headers={'User-agent': 'Mozilla/5.0'}).text,
                      attrs={"class":"table table-condensed table-bordered"})[0]
if np.any(t):
  print(t)
else:
  print('no data')

no data


In [0]:
import time

In [0]:
print('before')
time.sleep(3)
print('after')

before
after
